# Initiate
We first initiate the environment. **Use python 3.9 kenrel if running from jupyterlab**

In [1]:
import os
import bentoml
import mlflow
import darts 
#print(numpy.__version__)

from dotenv import load_dotenv
load_dotenv()


True

# Load model as bentos

Then we load the model from minio as a bento. In the link to the model we always use the pyfunc model folder. The bento name needs to be changed if a different model needs to be loaded, otherwise the new model will be considered a newer version of the old one.

In [2]:
model = bentoml.mlflow.import_model(
    "uc7_companies_no_covs", #bento model name 
    "s3://mlflow-bucket/40/2850a79365524979819f69e9bbd55514/artifacts/pyfunc_model", #minio uri
    signatures={'predict': {'batchable': True}})

bento_model = bentoml.mlflow.get(model.tag)
pyfunc_model: mlflow.pyfunc.PyFuncModel = bentoml.mlflow.load_model(
    model.tag)

2024/01/16 16:01:27 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - darts (current: 0.23.1, required: darts==0.22.0)
 - torch (current: 1.11.0, required: torch==1.11.0+cu102)
 - mlflow (current: 1.27.0, required: mlflow==1.25.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


Local path inside _load_pyfunc: /new_vol_300/marija/conda/multiusers/iccs/bentoml/models/uc7_companies_no_covs/ddzciyfuqcafv6qw/mlflow_model/data/2850a79365524979819f69e9bbd55514
Local path altered for loading: /new_vol_300/marija/conda/multiusers/iccs/bentoml/models/uc7_companies_no_covs/ddzciyfuqcafv6qw/mlflow_model/data/2850a79365524979819f69e9bbd55514
/new_vol_300/marija/conda/multiusers/iccs/bentoml/models/uc7_companies_no_covs/ddzciyfuqcafv6qw/mlflow_model/data/2850a79365524979819f69e9bbd55514
/new_vol_300/marija/conda/multiusers/iccs/bentoml/models/uc7_companies_no_covs/ddzciyfuqcafv6qw/mlflow_model/data/2850a79365524979819f69e9bbd55514

Loading local PKL model...


2024-01-16 16:01:44.647137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-16 16:01:47.104628: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 16:01:48.002182: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-16 16:01:48.002211: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Then we make sure that the model has been saved as a bento, and we get the tag of its latest version

In [4]:
print(bentoml.models.list("uc7_companies_no_covs")[0])

Model(tag="uc7_companies_no_covs:ddzciyfuqcafv6qw")


# Predict

To make sure that this model is providing predictions normaly, we test it using the series that is uploaded to minio

## LGBM

In [7]:
%%capture
# input example
input_example = {
            "n": "24",
            "series_uri": "s3://mlflow-bucket/40/b92fb3c675814fdebf08c23c7de9c001/artifacts/features/series.csv",
            "roll_size": "1",
            "future_covariates_uri": "None",
            "past_covariates_uri": "None",
            "batch_size": "1",
            "multiple": "True",
            "weather_covariates":[],
            "resolution": "60",
            "ts_id_pred": "BBB6030",
}
predictions = pyfunc_model.predict(input_example)

In [8]:
predictions

component,Value
Datetime,
2023-12-28 00:00:00,3.931897
2023-12-28 01:00:00,3.949497
2023-12-28 02:00:00,3.980993
2023-12-28 03:00:00,4.072373
2023-12-28 04:00:00,4.106251
2023-12-28 05:00:00,4.013561
2023-12-28 06:00:00,3.740512
2023-12-28 07:00:00,3.039017
2023-12-28 08:00:00,1.821295
